In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
#Define presto credentials
presto_host = 'presto.processing.yoda.run'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [3]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, drop_location_hex_8, drop_location_latitude, drop_location_longitude, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason
from orders.order_logs_snapshot
where yyyymmdd >= '20220919'
and yyyymmdd <= '20221016'
and city_name in ('Hyderabad')
and service_obj_service_name = 'Link'
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [4]:
#Create two dfs for dropped_in and dropped_out
df_copy['count'] = 1
df_dropped_in = df_copy.copy()
df_dropped_out = df_copy.copy()
df_dropped_in.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,count
0,632a270f7d42681b1c46c82c,1.807,8860a259bbfffff,17.443420,78.376907,Hyderabad,8860a24b51fffff,8860a259bbfffff,17.443420,78.376907,17.467382,78.365715,021815,02,0215,Link,02:00,3,20220921,dropped,None,1
1,632a28e19aa7cf28bdebd90a,1.900,8860a25b29fffff,17.399033,78.503494,Hyderabad,8860a25b2bfffff,8860a25b29fffff,17.399033,78.503494,17.401052,78.489250,022601,02,0215,Link,02:00,3,20220921,dropped,Taking longer than expected,1
2,632a2ca19aa7cf28bdebd9d6,0.962,8860b525b5fffff,17.356150,78.508430,Hyderabad,8860b525b5fffff,8860b525e7fffff,17.352946,78.535659,17.356142,78.508423,024201,02,0230,Link,02:59,3,20220921,customerCancelled,Wrong pickup location,1
3,632a22f09aa7cf28bdebd78f,0.707,8860a25903fffff,17.434853,78.449905,Hyderabad,8860a25903fffff,8860a259edfffff,17.440142,78.430342,17.434853,78.449905,020040,02,0200,Link,02:00,3,20220921,customerCancelled,Others,1
4,632a27a07ff9294e27586f45,0.261,8860b52c95fffff,17.435114,78.503087,Hyderabad,8860b52c91fffff,8860a259cdfffff,17.426403,78.426476,17.434086,78.502136,022040,02,0215,Link,02:00,3,20220921,customerCancelled,Change of plans,1


In [5]:
#Load the hexes to filter on
area1_hexes = pd.read_csv('/Users/rapido/Desktop/batching_demand/hyderabad/hyderabad_location_data_level1_demand_zone1.csv')
area2_hexes = pd.read_csv('/Users/rapido/Desktop/batching_demand/hyderabad/hyderabad_location_data_level1_demand_zone2.csv')

area1_hexes_list = list(area1_hexes['customer_location_hex_8'].unique())
area2_hexes_list = list(area2_hexes['customer_location_hex_8'].unique())

#Filter the dfs on the hexes identified for bangalore
df_dropped_in_area1 = df_dropped_in[df_dropped_in['drop_location_hex_8'].isin(area1_hexes_list)]
df_dropped_in_area1 = df_dropped_in_area1[df_dropped_in_area1['order_status'] == 'dropped']
df_dropped_in_area2 = df_dropped_in[df_dropped_in['drop_location_hex_8'].isin(area2_hexes_list)]
df_dropped_in_area2 = df_dropped_in_area2[df_dropped_in_area2['order_status'] == 'dropped']


df_dropped_out_area1 = df_dropped_out[df_dropped_out['customer_location_hex_8'].isin(area1_hexes_list)]
df_dropped_out_area1 = df_dropped_out_area1[df_dropped_out_area1['order_status'] == 'dropped']
df_dropped_out_area2 = df_dropped_out[df_dropped_out['customer_location_hex_8'].isin(area2_hexes_list)]
df_dropped_out_area2 = df_dropped_out_area2[df_dropped_out_area2['order_status'] == 'dropped']

In [6]:
#Groupby on date and hour to get the number of rides dropped_in/out
df_dropped_in_area1_rides = df_dropped_in_area1.groupby(['yyyymmdd', 'hour'])['count'].sum().reset_index()
df_dropped_in_area1_rides = df_dropped_in_area1_rides.rename(columns = {'count':'dropped_rides_in'})
df_dropped_in_area2_rides = df_dropped_in_area2.groupby(['yyyymmdd', 'hour'])['count'].sum().reset_index()
df_dropped_in_area2_rides = df_dropped_in_area2_rides.rename(columns = {'count':'dropped_rides_in'})

df_dropped_out_area1_rides = df_dropped_out_area1.groupby(['yyyymmdd', 'hour'])['count'].sum().reset_index()
df_dropped_out_area1_rides = df_dropped_out_area1_rides.rename(columns = {'count':'dropped_rides_out'})
df_dropped_out_area2_rides = df_dropped_out_area2.groupby(['yyyymmdd', 'hour'])['count'].sum().reset_index()
df_dropped_out_area2_rides = df_dropped_out_area2_rides.rename(columns = {'count':'dropped_rides_out'})

In [7]:
#Merge dropped_in and dropped_out
df_replenishment_area1 = df_dropped_in_area1_rides.merge(df_dropped_out_area1_rides, on = ['yyyymmdd', 'hour'], how = 'left')
df_replenishment_area2 = df_dropped_in_area2_rides.merge(df_dropped_out_area2_rides, on = ['yyyymmdd', 'hour'], how = 'left')

df_replenishment_area1 = df_replenishment_area1.sort_values(by = ['yyyymmdd', 'hour'], ascending=True)
df_replenishment_area2 = df_replenishment_area2.sort_values(by = ['yyyymmdd', 'hour'], ascending=True)

#Calculate replenishment
df_replenishment_area1['replenishment'] = df_replenishment_area1['dropped_rides_in'] - df_replenishment_area1['dropped_rides_out']
df_replenishment_area2['replenishment'] = df_replenishment_area2['dropped_rides_in'] - df_replenishment_area2['dropped_rides_out']
df_replenishment_area1.head()

,yyyymmdd,hour,dropped_rides_in,dropped_rides_out,replenishment
0,20220919,00,156,173,-17
1,20220919,01,66,97,-31
2,20220919,02,59,58,1
3,20220919,03,33,37,-4
4,20220919,04,47,33,14


In [8]:
#Save the replenishment to excel
df_replenishment_area1.to_csv('replenishment_hyderabad_area1.csv', index = False)
df_replenishment_area2.to_csv('replenishment_hyderabad_area2.csv', index = False)